# JJ/SQUID Flux-Pumped Lattice — πₐ/ARP Sidebands
This notebook maps a flux pump \(\Phi(t)=\Phi_{dc}+\Phi_{ac}\sin\Omega t\) through a SQUID:
\( I_c(\Phi)=2 I_0 |\cos(\pi \Phi/\Phi_0)| \Rightarrow L_J(\Phi)=\Phi_0/(2\pi I_c) \).
With a transmission line section, the per‑unit‑length inductance is \(L(x,t)=L_{par}+L_J(\Phi(t))\) (assuming constant \(C\)).
To first order, \(\delta n/n \approx \tfrac12\, \delta L/L\) and the phase modulation index is \(\beta \approx (\omega_0 \tau)\,(\delta n/n)\).
First sidebands: \(P_{\pm1}/P_0 \approx [J_1(\beta)/J_0(\beta)]^2\).

In [ ]:
import numpy as np, matplotlib.pyplot as plt
from mpmath import besselj
Phi0 = 2.067833848e-15
I0 = 3e-06
C_line = 1.2e-10
L_par = 2.2e-10
Lseg = 0.01
f0 = 6000000000.0
def Ic_squid(phi): return 2*I0*abs(np.cos(np.pi*phi/Phi0))
def LJ(phi): 
    Ic = Ic_squid(phi)
    Ic = np.maximum(Ic, 1e-12)
    return Phi0/(2*np.pi*Ic)
def vg(L,C): return 1/np.sqrt(L*C)
def delta_n_over_n(phi_dc, phi_ac):
    dphi=1e-18
    L0=L_par+LJ(phi_dc)
    dLdphi=(LJ(phi_dc+dphi)-LJ(phi_dc-dphi))/(2*dphi)
    m=(dLdphi/L0)*phi_ac
    return 0.5*m
def beta_from_delta_n(dn, v, Lseg, f0): 
    tau=Lseg/v; return 2*np.pi*f0*tau*dn


## SQUID $I_c(\Phi)$ and $L_J(\Phi)$

In [ ]:
phi = np.linspace(0, 0.5, 400)*Phi0
Ic = [Ic_squid(p) for p in phi]
LJv= [LJ(p) for p in phi]
fig,ax=plt.subplots(1,2,figsize=(10,4))
ax[0].plot(phi/Phi0, np.array(Ic)/1e-6); ax[0].set_title("Ic vs Φ/Φ0"); ax[0].set_xlabel("Φ/Φ0"); ax[0].set_ylabel("μA")
ax[1].plot(phi/Phi0, np.array(LJv)*1e9); ax[1].set_title("LJ vs Φ/Φ0"); ax[1].set_xlabel("Φ/Φ0"); ax[1].set_ylabel("nH")
plt.show()

## Sideband sweep vs pump amplitude at different DC biases

In [ ]:
phi_dc_list=[0.20,0.30,0.45]
phi_ac_vals=np.linspace(0,0.2,200)
plt.figure(figsize=(7.2,4.4))
for phi_dc_frac in phi_dc_list:
    vg0=vg(L_par+LJ(phi_dc_frac*Phi0), C_line)
    ratios=[]
    for phi_ac_frac in phi_ac_vals:
        dn=delta_n_over_n(phi_dc_frac*Phi0, phi_ac_frac*Phi0)
        beta=beta_from_delta_n(dn, vg0, Lseg, f0)
        J0=float(besselj(0,beta)); J1=float(besselj(1,beta))
        r=(J1/J0)**2 if J0!=0 else np.nan
        ratios.append(10*np.log10(r+1e-30))
    plt.plot(phi_ac_vals, ratios, label=f"Φdc/Φ0={phi_dc_frac:.2f}")
plt.xlabel("Φac/Φ0"); plt.ylabel("P±1/P0 (dBc)"); plt.legend(); plt.title("Sideband level vs flux pump")
plt.show()

## Loss impact (uniform insertion loss across line)

In [ ]:
loss_dB_vals=np.linspace(0,6,25)
phi_dc_demo=0.30; phi_ac_demo=0.10
vg0=vg(L_par+LJ(phi_dc_demo*Phi0), C_line)
dn=delta_n_over_n(phi_dc_demo*Phi0, phi_ac_demo*Phi0)
beta=beta_from_delta_n(dn, vg0, Lseg, f0)
J0=float(besselj(0,beta)); J1=float(besselj(1,beta))
carrier=[]; sb1=[]; ratio=[]
for LdB in loss_dB_vals:
    A=10**(-LdB/20)
    P0=(J0*A)**2; P1=(J1*A)**2
    carrier.append(10*np.log10(P0+1e-30)); sb1.append(10*np.log10(P1+1e-30)); ratio.append(10*np.log10((P1/P0)+1e-30))
import matplotlib.pyplot as plt
fig,ax=plt.subplots(1,2,figsize=(11,4))
ax[0].plot(loss_dB_vals, carrier, label="Carrier"); ax[0].plot(loss_dB_vals, sb1, label="±1 sideband")
ax[0].set_xlabel("Insertion loss (dB)"); ax[0].set_ylabel("Abs power (dB rel.)"); ax[0].legend(); ax[0].set_title("Absolute levels")
ax[1].plot(loss_dB_vals, ratio); ax[1].set_xlabel("Insertion loss (dB)"); ax[1].set_ylabel("P±1/P0 (dBc)"); ax[1].set_title("Ratio (robust)")
plt.show()